In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import ast
import os
import re

In [70]:
def load_dataset(f_path):
    df = pd.read_csv(f_path)

    # if f_path.__contains__("5_Modular"):
    #     df["faithfulness"] = df["faithfulness"].apply(lambda x: float(ast.literal_eval(x)[0]))
    #     df["answer_relevancy"] = df["answer_relevancy"].apply(extract_np_float)

    #     return df

    # faithfulness, answer_relevancy list -> float 수치형 데이터 변환
    df["faithfulness"] = df["faithfulness"].astype("float")
    df["answer_relevancy"] = df["answer_relevancy"].astype("float")

    return df

# ✅ 정규표현식: 괄호 안 숫자만 추출
def extract_np_float(s):
    try:
        # 리스트 괄호까지 포함된 문자열에서도 추출 가능
        match = re.search(r"np\.float64\(([\d\.]+)\)", s)
        if match:
            return float(match.group(1))
        else:
            return None
    except Exception as e:
        return None

In [ ]:
dir_path = ""
dir_path2 = ""

df1 = load_dataset(dir_path + '1_QAset_NaiveRAG_ScoreMethod_aggregated_docs_v3.csv')
df2_li = load_dataset(dir_path + '2_QAset_ModularRAG_ScoreMethod_Literature_aggregated_docs_v3.csv')
df2_sc = load_dataset(dir_path + '2_QAset_ModularRAG_ScoreMethod_Science_aggregated_docs_v3.csv')
df3_ge = load_dataset(dir_path + '3_QAset_ModularRAG_ActionMethod_General_aggregated_docs_v3.csv')
df3_li = load_dataset(dir_path + '3_QAset_ModularRAG_ActionMethod_Literature_aggregated_docs_v3.csv')
df3_sc = load_dataset(dir_path + '3_QAset_ModularRAG_ActionMethod_Science_aggregated_docs_v3.csv')
df5_ge = load_dataset("/Users/sarahpark/Library/Mobile Documents/com~apple~CloudDocs/데이터분석/AIFFEL_Reserach/Aiffelton/Dataset/Evaluation/Results/5_Modular_RAG_general_QA_final_aggregated_docs_v4.csv")
df5_li = load_dataset(dir_path2 + "5_Modular_RAG_literature_QA_final_aggregated_docs_v3.csv")
df5_sc = load_dataset(dir_path2 + "5_Modular_RAG_science_QA_final_aggregated_docs_v3.csv")

### 각 데이터별 faithfulness, answer_relevancy 평균값 추출

In [72]:
def summarize_metrics(df, model_name, persona_name):
    faith = df["faithfulness"]
    ans_rel = df["answer_relevancy"]

    return {
        "model": model_name,
        "persona": persona_name,

        "faithfulness_avg": faith.mean(),
        "faithfulness_avg_wo_zeros": faith[faith != 0].mean(),
        
        "answer_relevancy_avg": ans_rel.mean(),
        "answer_relevancy_wo_zeros": ans_rel[ans_rel != 0].mean(),

        "faithfulness_zero_cnt": (faith == 0).sum(),
        "answer_relevancy_zero_cnt": (ans_rel == 0).sum(),
        "total_single_turn_responses": len(df)
    }

In [73]:
summary_data = []

summary_data.append(summarize_metrics(df1, "model1", "general"))
summary_data.append(summarize_metrics(df2_li, 'model2', 'literature'))
summary_data.append(summarize_metrics(df2_sc, 'model2', 'science'))
summary_data.append(summarize_metrics(df3_ge, 'model3', 'general'))
summary_data.append(summarize_metrics(df3_li, 'model3', 'literature'))
summary_data.append(summarize_metrics(df3_sc, 'model3', 'science'))
summary_data.append(summarize_metrics(df5_ge, "model5", "general"))
summary_data.append(summarize_metrics(df5_li, "model5", "literature"))
summary_data.append(summarize_metrics(df5_sc, "model5", "science"))

summary_df = pd.DataFrame(summary_data)
summary_df["non-single turn responses"] = 60 - summary_df["total_single_turn_responses"]

summary_df

,model,persona,faithfulness_avg,faithfulness_avg_wo_zeros,answer_relevancy_avg,answer_relevancy_wo_zeros,faithfulness_zero_cnt,answer_relevancy_zero_cnt,total_single_turn_responses,non-single turn responses
0,model1,general,0.755560,0.755560,0.748921,0.775669,0,2,58,2
1,model2,literature,0.814725,0.814725,0.747708,0.795434,0,3,50,10
2,model2,science,0.873237,0.873237,0.622534,0.737211,0,7,45,15
3,model3,general,0.822198,0.822198,0.746421,0.785706,0,3,60,0
4,model3,literature,0.829370,0.829370,0.710613,0.789570,0,6,60,0
5,model3,science,0.857353,0.857353,0.744866,0.798071,0,4,60,0
6,model5,general,0.879678,0.879678,0.749301,0.776062,0,2,58,2
7,model5,literature,0.834509,0.834509,0.431356,0.727142,0,24,59,1
8,model5,science,0.843550,0.843550,0.444310,0.731805,0,22,56,4


In [74]:
summary_df.sort_values(by = "answer_relevancy_wo_zeros", ascending= False)

,model,persona,faithfulness_avg,faithfulness_avg_wo_zeros,answer_relevancy_avg,answer_relevancy_wo_zeros,faithfulness_zero_cnt,answer_relevancy_zero_cnt,total_single_turn_responses,non-single turn responses
5,model3,science,0.857353,0.857353,0.744866,0.798071,0,4,60,0
1,model2,literature,0.814725,0.814725,0.747708,0.795434,0,3,50,10
4,model3,literature,0.829370,0.829370,0.710613,0.789570,0,6,60,0
3,model3,general,0.822198,0.822198,0.746421,0.785706,0,3,60,0
6,model5,general,0.879678,0.879678,0.749301,0.776062,0,2,58,2
0,model1,general,0.755560,0.755560,0.748921,0.775669,0,2,58,2
2,model2,science,0.873237,0.873237,0.622534,0.737211,0,7,45,15
8,model5,science,0.843550,0.843550,0.444310,0.731805,0,22,56,4
7,model5,literature,0.834509,0.834509,0.431356,0.727142,0,24,59,1


In [75]:
summary_df["faithfulness_avg_adj"] = summary_df["faithfulness_avg"] * summary_df["total_single_turn_responses"] / 60
summary_df["answer_relevancy_avg_adj"] = summary_df["answer_relevancy_avg"] * summary_df["total_single_turn_responses"] / 60
summary_df

,model,persona,faithfulness_avg,faithfulness_avg_wo_zeros,answer_relevancy_avg,answer_relevancy_wo_zeros,faithfulness_zero_cnt,answer_relevancy_zero_cnt,total_single_turn_responses,non-single turn responses,faithfulness_avg_adj,answer_relevancy_avg_adj
0,model1,general,0.755560,0.755560,0.748921,0.775669,0,2,58,2,0.730374,0.723957
1,model2,literature,0.814725,0.814725,0.747708,0.795434,0,3,50,10,0.678937,0.623090
2,model2,science,0.873237,0.873237,0.622534,0.737211,0,7,45,15,0.654928,0.466900
3,model3,general,0.822198,0.822198,0.746421,0.785706,0,3,60,0,0.822198,0.746421
4,model3,literature,0.829370,0.829370,0.710613,0.789570,0,6,60,0,0.829370,0.710613
5,model3,science,0.857353,0.857353,0.744866,0.798071,0,4,60,0,0.857353,0.744866
6,model5,general,0.879678,0.879678,0.749301,0.776062,0,2,58,2,0.850355,0.724324
7,model5,literature,0.834509,0.834509,0.431356,0.727142,0,24,59,1,0.820601,0.424166
8,model5,science,0.843550,0.843550,0.444310,0.731805,0,22,56,4,0.787313,0.414690


In [86]:
fig = px.bar(
    summary_df,
    x = "model",
    y = "faithfulness_avg",
    color = "persona",
    barmode = "group",
    title = "Avg Faithfulness by Model",
    category_orders = {"model": ["model1", "model2", "model3"]},
    range_y = [0, 1]
)

fig.show()

In [87]:
fig = px.bar(
    summary_df,
    x = "model",
    y = "faithfulness_avg_adj",
    color = "persona",
    barmode = "group",
    title = "Avg Faithfulness by Model",
    category_orders = {"model": ["model1", "model2", "model3"]},
    range_y = [0, 1]
)

fig.show()

In [76]:
fig = px.bar(
    summary_df,
    x = "model",
    y = "faithfulness_avg_adj",
    color = "persona",
    barmode = "group",
    title = "Avg Faithfulness by Model with Zero Values",
    category_orders = {"model": ["model1", "model2", "model3"]},
    range_y = [0, 1]
)

fig.show()

In [77]:
fig = px.bar(
    summary_df,
    x = "model",
    y = "faithfulness_avg_wo_zeros",
    color = "persona",
    barmode = "group",
    title = "Avg Faithfulness by Model with Zero Values",
    category_orders = {"model": ["model1", "model2", "model3"]},
    range_y = [0, 1]
)

fig.show()

In [78]:
fig = px.bar(
    summary_df,
    x = "model",
    y = "answer_relevancy_avg_adj",
    color = "persona",
    barmode = "group",
    title = "Avg Answer Relevancy by Model with Zero Values",
    category_orders = {"model": ["model1", "model2", "model3"]},
    range_y = [0, 1]
)

fig.show()

In [90]:
fig = px.bar(
    summary_df,
    x = "model",
    y = "answer_relevancy_avg",
    color = "persona",
    barmode = "group",
    title = "Avg Answer Relevancy",
    category_orders = {"model": ["model1", "model2", "model3"]},
    range_y = [0, 1]
)

fig.show()

In [79]:
summary_df.columns

Index(['model', 'persona', 'faithfulness_avg', 'faithfulness_avg_wo_zeros',
       'answer_relevancy_avg', 'answer_relevancy_wo_zeros',
       'faithfulness_zero_cnt', 'answer_relevancy_zero_cnt',
       'total_single_turn_responses', 'non-single turn responses',
       'faithfulness_avg_adj', 'answer_relevancy_avg_adj'],
      dtype='object')

In [80]:
fig = px.bar(
    summary_df,
    x = "model",
    y = "answer_relevancy_wo_zeros",
    color = "persona",
    barmode = "group",
    title = "Avg Answer Relevancy by Model without Zero Values",
    category_orders = {"model": ["model1", "model2", "model3"]},
    range_y = [0, 1]
)

fig.show()

In [81]:
fig = px.bar(
    summary_df,
    x = "model",
    y = "answer_relevancy_zero_cnt",
    color = "persona",
    barmode = "group",
    title = "Zero values of Answer Relevancy by Model",
    category_orders = {"model": ["model1", "model2", "model3"]},
)

fig.show()

In [82]:
fig = px.bar(
    summary_df,
    x = "model",
    y = "non-single turn responses",
    color = "persona",
    barmode = "group",
    title = "Non Single Turn Responses by Model",
    category_orders = {"model": ["model1", "model2", "model3"]},
)

fig.show()

# 여러 지표 비교

In [83]:
import plotly.graph_objects as go

fig = go.Figure()

# faithfulness
fig.add_trace(go.Bar(
    x=summary_df["persona"],
    y=summary_df["faithfulness_avg"],
    name="Faithfulness",
    marker_color='blue'
))

# answer_relevancy
fig.add_trace(go.Bar(
    x=summary_df["persona"],
    y=summary_df["answer_relevancy_avg"],
    name="Answer Relevancy",
    marker_color='green'
))

fig.update_layout(
    title="페르소나별 평균 성능 비교",
    barmode="group",
    xaxis_title="Persona",
    yaxis_title="Score",
)
fig.show()

In [84]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)  